# BERT4Rec

## Modules

In [2]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

/home/ksh/miniconda3/envs/modeling/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.is_available()

True

## Hyper-parameters

In [4]:
# model setting
max_len = 20
hidden_units = 50
num_heads = 1
num_layers = 2
dropout_rate=0.5
num_workers = 1
device = 'cuda'

# training setting
lr = 0.001
batch_size = 128
num_epochs = 200
mask_prob = 0.15 # for cloze task

params = {
    "max_len":20,
    "hidden_units":50,
    "num_heads":1,
    "num_layers":2,
    "dropout_rate":0.5,
    "device":'cuda',
}

## Data preprocessing

Data preprocessing은 SASRec과 달리 cloze task 수행을 위한 masking을 추가해주어야 합니다. 그 외의 부분은 SASRec과 유사합니다. Masking이 추가 되었다는 것에 주의하여 Data preprocessing을 구현해봅시다.

In [1]:
############# 중요 #############
# data_path는 사용자의 디렉토리에 맞게 설정해야 합니다.
FOLDER_PATH = "./data"

FILE_NAME = "bert.csv"
FILE_PATH = os.path.join(FOLDER_PATH, FILE_NAME)

df = pd.read_csv(FILE_PATH)

NameError: name 'os' is not defined

In [6]:
item_ids = df['item'].unique()
user_ids = df['user'].unique()
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // batch_size

# user, item indexing
item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item), num_item+1: mask idx
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
df.sort_values(['user_idx', 'time'], inplace=True)
del df['item'], df['user']

In [18]:
num_item

8587

In [7]:
df["user_idx"].unique()

array([   0,    1,    2, ..., 2575, 2576, 2577])

In [8]:
# train set, valid set 생성
users = defaultdict(list) # defaultdict은 dictionary의 key가 없을때 default 값을 value로 반환
user_train = {}
user_valid = {}
for u, i, t in zip(df['user_idx'], df['item_idx'], df['time']):
    users[u].append(i)

for user in users:
    user_train[user] = users[user][:-1]
    user_valid[user] = [users[user][-1]]

print(f'num users: {num_user}, num items: {num_item}')

params["num_item"] = num_item

num users: 2578, num items: 8587


SASRec에서는 sample function을 따로 구현했지만, BERT4Rec에서는 pytorch의 내장 data loader를 사용하여서 구현해봅시다. Custom dataset을 구현하기 위해서는 pytorch의 dataset class를 상속받아 생성한 class의 구현이 필요합니다. 자세한 문법은 https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 에서 확인할 수 있습니다.

In [9]:
class SeqDataset(Dataset):
    def __init__(self, user_train, num_user, num_item, max_len, mask_prob):
        self.user_train = user_train
        self.num_user = num_user
        self.num_item = num_item
        self.max_len = max_len
        self.mask_prob = mask_prob

    def __len__(self):
        # 총 user의 수 = 학습에 사용할 sequence의 수
        return self.num_user

    def __getitem__(self, user):
        # iterator를 구동할 때 사용됩니다.
        seq = self.user_train[user]
        tokens = []
        labels = []
        for s in seq:
            prob = np.random.random() # TODO1: numpy를 사용해서 0~1 사이의 임의의 값을 샘플링하세요.
            if prob < self.mask_prob:
                prob /= self.mask_prob

                # BERT 학습
                if prob < 0.8:
                    # masking
                    tokens.append(self.num_item + 1)  # mask_index: num_item + 1, 0: pad, 1~num_item: item index
                elif prob < 0.9:
                    tokens.append(np.random.randint(1, self.num_item+1))  # item random sampling
                else:
                    tokens.append(s)
                labels.append(s)  # 학습에 사용
            else:
                tokens.append(s)
                labels.append(0)  # 학습에 사용 X, trivial
        tokens = tokens[-self.max_len:]
        labels = labels[-self.max_len:]
        mask_len = self.max_len - len(tokens)

        # zero padding
        tokens = [0] * mask_len + tokens
        labels = [0] * mask_len + labels
        return torch.LongTensor(tokens), torch.LongTensor(labels)

## Model

Multi-head attention 부분은 SASRec과 같습니다. 다만 point-wise feed forward network에서 GeLU activation을 사용한다는 차이점이 있습니다. 그리고 point-wise feed forward network의 MLP의 dimension을 SASRec에서는 일정하게 유지시켰지만 BERT4Rec에서는 NLP에서 사용하는 모델과 같이 dimension에 4배 차이를 두었습니다. Positional encoding 역시 SASRec과 마찬가지로 학습 가능하도록 모델링합니다.

In [10]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate) # dropout rate

    def forward(self, Q, K, V, mask):
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)
        attn_score = attn_score.masked_fill(mask == 0, -1e9)  # 유사도가 0인 지점은 -infinity로 보내 softmax 결과가 0이 되도록 함
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # dim of output : batchSize x num_head x seqLen x hidden_units
        return output, attn_dist

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head의 수
        self.hidden_units = hidden_units
        self.hidden_units_per_head = self.hidden_units // self.num_heads

        # query, key, value, output 생성을 위해 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate) # scaled dot product attention module을 사용하여 attention 계산
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, enc, mask):
        residual = enc # residual connection을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        
        # Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        # K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        # V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units_per_head)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units_per_head)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units_per_head)

        # Head별로 각기 다른 attention이 가능하도록 Transpose 후 각각 attention에 통과시킴
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, seqlen, -1)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist

class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        # SASRec과의 dimension 차이가 있습니다.
        self.W_1 = nn.Linear(hidden_units, 4 * hidden_units)
        self.W_2 = nn.Linear(4 * hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        output = self.W_2(F.gelu(self.dropout(self.W_1(x)))) # activation: relu -> gelu
        output = self.layerNorm(self.dropout(output) + residual)
        return output

class BERT4RecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(BERT4RecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

### BERT4Rec

위에서 구현한 class를 가지고 BERT4Rec을 구현해봅시다. BERT4Rec의 item embedding은 item의 개수에 비해 2개 더 많게 설정해야 합니다. 이는 padding과 cloze task를 위한 mask를 표기하기 위함입니다. 최종적으로 다음 item을 예측할 때도 전체 log에 mask index를 추가하여 예측을 수행합니다.

In [11]:
# model setting
class BERT4Rec(nn.Module):
    def __init__(self, **params):
        super(BERT4Rec, self).__init__()
        
        self.num_item = params["num_item"]
        self.hidden_units = params["hidden_units"]
        self.num_heads = params["num_heads"]
        self.num_layers = params["num_layers"]
        self.max_len = params["max_len"]
        self.dropout_rate = params["dropout_rate"]
        self.device = params["device"]

        self.item_emb = nn.Embedding(self.num_item + 2, self.hidden_units, padding_idx=0) # TODO2: mask와 padding을 고려하여 embedding을 생성해보세요.
        self.pos_emb = nn.Embedding(self.max_len, self.hidden_units) # learnable positional encoding
        self.dropout = nn.Dropout(self.dropout_rate)
        self.emb_layernorm = nn.LayerNorm(self.hidden_units, eps=1e-6)

        self.blocks = nn.ModuleList([BERT4RecBlock(self.num_heads, self.hidden_units, self.dropout_rate) for _ in range(self.num_layers)])
        self.out = nn.Linear(self.hidden_units, self.num_item + 1) # TODO3: 예측을 위한 output layer를 구현해보세요. (num_item 주의)

    def forward(self, log_seqs):
        seqs = self.item_emb(torch.LongTensor(log_seqs).to(self.device))
        positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))
        seqs = self.emb_layernorm(self.dropout(seqs))

        mask = torch.BoolTensor(log_seqs > 0).unsqueeze(1).repeat(1, log_seqs.shape[1], 1).unsqueeze(1).to(self.device) # mask for zero pad
        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        out = self.out(seqs)
        return out

## Training

In [12]:
print(params)

{'max_len': 20, 'hidden_units': 50, 'num_heads': 1, 'num_layers': 2, 'dropout_rate': 0.5, 'device': 'cuda', 'num_item': 8587}


In [13]:
model = BERT4Rec(**params)
model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0) # label이 0인 경우 무시
seq_dataset = SeqDataset(user_train, num_user, num_item, max_len, mask_prob)
data_loader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True) # TODO4: pytorch의 DataLoader와 seq_dataset을 사용하여 학습 파이프라인을 구현해보세요.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [14]:
def recall_and_precision_at_k(user_true_items, user_predicted_items, k=10):
    num_hits = 0
    num_true_items = 0
    num_predicted_items = 0

    for user in user_true_items.keys():
        true_items = set(user_true_items[user])
        predicted_items = set(user_predicted_items[user][:k])

        num_hits += len(true_items & predicted_items)
        num_true_items += len(true_items)
        num_predicted_items += len(predicted_items)

    recall = num_hits / float(num_true_items) if num_true_items > 0 else 0
    precision = num_hits / float(num_predicted_items) if num_predicted_items > 0 else 0
    return recall, precision

def f1_score(recall, precision):
    return 2 * (precision * recall) / (precision + recall+1e-8) 

def evaluation():
    def random_neg(l, r, s):
        t = np.random.randint(l, r)
        while t in s:
            t = np.random.randint(l, r)
        return t

    model.eval()

    NDCG = 0.0
    HIT = 0.0
    total_recall = 0.0
    total_precision = 0.0

    num_user_sample = num_user
    users = range(0, num_user)
    k= 10
    for u in users:
        seq = (user_train[u])[-max_len:] 
        rated = set(user_train[u] + user_valid[u])
        neg = list((set(item_ids)-set(rated)))
        # 아이템 하나 당 negative sampling의 개수 지정
        item_idx = [user_valid[u][0]] + [random_neg(0, len(item_ids), rated) for _ in range(45)]
        
        seq_tensor = torch.tensor(seq, dtype=torch.long).unsqueeze(0)
        item_idx_tensor = torch.tensor(item_idx, dtype=torch.long)

        with torch.no_grad():
            predictions = -model(seq_tensor)
            predictions = predictions[0, -1, item_idx_tensor]
            rank = predictions.argsort().argsort()[0].item()

        if rank < k:
            NDCG += 1 / np.log2(rank + 2)
            HIT += 1
            total_recall += 1 / min(len(user_valid[u]), k)
            total_precision += 1 / k

    recall = total_recall / num_user_sample
    precision = total_precision / num_user_sample
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)

    print(f'NDCG@10: {NDCG/num_user_sample}')
    print(f'Recall@10: {recall}')
    print(f'Precision@10: {precision}')
    print(f'F1: {f1_score}')


In [15]:
# mlflow.set_tracking_uri(uri="http://101.79.11.75:8010")
# mlflow.set_experiment("bert4rec")
# mlflow.pytorch.autolog()

In [16]:
with mlflow.start_run() as run:
    # mlflow.log_param(key="max_len", value=max_len)
    # mlflow.log_param(key="hidden_units", value=50)
    # mlflow.log_param(key="num_heads", value=1)
    # mlflow.log_param(key="num_layers", value=2)
    # mlflow.log_param(key="dropout_rate", value=0.5)
    # mlflow.log_param(key="num_workers", value=1)
    # mlflow.log_param(key="device", value='cuda')
    # mlflow.log_param(key="lr", value=0.001)
    # mlflow.log_param(key="batch_size", value=128)
    # mlflow.log_param(key="num_epochs", value=200)
    # mlflow.log_param(key="mask_prob", value=0.15)
    
    for epoch in range(1, num_epochs + 1):
        tbar = tqdm(data_loader)
        for step, (log_seqs, labels) in enumerate(tbar):
            model.train()
            logits = model(log_seqs)

            # size matching
            logits = logits.view(-1, logits.size(-1))
            labels = labels.view(-1).to(device)

            optimizer.zero_grad()
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            tbar.set_description(f'Epoch: {epoch:3d}| Step: {step:3d}| Train loss: {loss:.5f}')
            mlflow.log_metric('loss', loss)
    evaluation()
    
    
    # from mlflow.types import Schema, TensorSpec
    
    # input_schema = Schema([TensorSpec(np.dtype(np.int32), (-1, 20))])
    # output_schema = Schema([TensorSpec(np.dtype(np.int32), (20, 8588))])
    # signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    
    # mlflow.pytorch.log_model(model, "bert4rec")

Epoch:   1| Step:  20| Train loss: 9.07810: 100%|██████████| 21/21 [00:00<00:00, 24.66it/s]
Epoch:   2| Step:  20| Train loss: 8.42594: 100%|██████████| 21/21 [00:00<00:00, 72.21it/s]
Epoch:   3| Step:  20| Train loss: 7.85969: 100%|██████████| 21/21 [00:00<00:00, 75.64it/s]
Epoch:   4| Step:  20| Train loss: 7.91114: 100%|██████████| 21/21 [00:00<00:00, 75.96it/s]
Epoch:   5| Step:  20| Train loss: 6.96334: 100%|██████████| 21/21 [00:00<00:00, 75.65it/s]
Epoch:   6| Step:  20| Train loss: 5.50122: 100%|██████████| 21/21 [00:00<00:00, 74.46it/s]
Epoch:   7| Step:  20| Train loss: 8.44214: 100%|██████████| 21/21 [00:00<00:00, 75.92it/s]
Epoch:   8| Step:  20| Train loss: 7.94709: 100%|██████████| 21/21 [00:00<00:00, 75.77it/s]
Epoch:   9| Step:  20| Train loss: 6.92905: 100%|██████████| 21/21 [00:00<00:00, 72.15it/s]
Epoch:  10| Step:  20| Train loss: 8.82403: 100%|██████████| 21/21 [00:00<00:00, 75.94it/s]
Epoch:  11| Step:  20| Train loss: 8.81872: 100%|██████████| 21/21 [00:00<00:00,

NDCG@10: 0.5109266653023485| HIT@10: 0.591


### Evaluation

In [15]:
evaluation()

NDCG@10: 0.5256629085257893| HIT@10: 0.61


In [19]:
torch.save(model, "/home/ksh/GiftHub_model/modeling/amazon_cf/model/model.pt")

In [20]:
# model_1 = BERT4Rec(**params)
# model_1.load_state_dict(torch.load("/home/ksh/GiftHub_model/modeling/amazon_cf/model/model.pt"))
model_1 = torch.load("/home/ksh/GiftHub_model/modeling/amazon_cf/model/model.pt")
model_1.to(device)

BERT4Rec(
  (item_emb): Embedding(8589, 50, padding_idx=0)
  (pos_emb): Embedding(20, 50)
  (dropout): Dropout(p=0.5, inplace=False)
  (emb_layernorm): LayerNorm((50,), eps=1e-06, elementwise_affine=True)
  (blocks): ModuleList(
    (0): BERT4RecBlock(
      (attention): MultiHeadAttention(
        (W_Q): Linear(in_features=50, out_features=50, bias=False)
        (W_K): Linear(in_features=50, out_features=50, bias=False)
        (W_V): Linear(in_features=50, out_features=50, bias=False)
        (W_O): Linear(in_features=50, out_features=50, bias=False)
        (attention): ScaledDotProductAttention(
          (dropout): Dropout(p=0.5, inplace=False)
        )
        (dropout): Dropout(p=0.5, inplace=False)
        (layerNorm): LayerNorm((50,), eps=1e-06, elementwise_affine=True)
      )
      (pointwise_feedforward): PositionwiseFeedForward(
        (W_1): Linear(in_features=50, out_features=200, bias=True)
        (W_2): Linear(in_features=200, out_features=50, bias=True)
        (d

In [21]:
# 인덱스로 변경된 Raw Data 세팅 (user_df)
users_dic = defaultdict(list)
user_df = {}
for u, i, t in zip(df['user_idx'], df['item_idx'], df['time']):
    users_dic[u].append(i)
    
for user in users_dic:
    user_df[user] = users_dic[user]

In [54]:
user_df[0]
# used_items_list = [a - 1 for a in user_df[0]]
# used_items_list

[1, 2, 3, 4, 5, 6]

In [ ]:
# inference
model.eval()
predict_list = []
for u in tqdm(range(num_user)):
    seq = (user_df[u])[-max_len:]
    used_items_list = [a - 1 for a in user_df[u]]  # 사용한 아이템에 대해 인덱스 계산을 위해 1씩 뺀다.
    
    if len(seq) < max_len:
        seq = np.pad(seq, (max_len - len(seq), 0), 'constant', constant_values=0)  # 패딩 추가

    with torch.no_grad():
        predictions = -model(np.array([seq]))
        predictions = predictions[0][-1][1:]  # mask 제외
        predictions[used_items_list] = np.inf  # 사용한 아이템은 제외하기 위해 inf
        rank = predictions.argsort().argsort().tolist()
        
        for i in range(10):
            rank.index(i)
            predict_list.append([u, rank.index(i)])

100%|██████████| 2578/2578 [00:07<00:00, 331.99it/s]


In [ ]:
seq

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 8580, 8581,
       6237, 8582, 4503, 8583, 8584, 8585, 8586, 8587, 8588])

In [22]:
model(np.array([np.array([0,0,0,0,0,0,0,0,0,5,1,10,15,67,96,84,635,8096,211,182])]))[0].shape

torch.Size([20, 8588])

In [ ]:
predict_list

[[0, 60],
 [0, 1748],
 [0, 6702],
 [0, 2395],
 [0, 3787],
 [0, 5554],
 [0, 5167],
 [0, 7019],
 [0, 8546],
 [0, 4832],
 [1, 1748],
 [1, 7019],
 [1, 2395],
 [1, 2792],
 [1, 638],
 [1, 113],
 [1, 8546],
 [1, 5949],
 [1, 7994],
 [1, 482],
 [2, 6550],
 [2, 283],
 [2, 699],
 [2, 1748],
 [2, 638],
 [2, 6927],
 [2, 2792],
 [2, 6702],
 [2, 1990],
 [2, 5008],
 [3, 1098],
 [3, 4114],
 [3, 739],
 [3, 360],
 [3, 699],
 [3, 306],
 [3, 697],
 [3, 302],
 [3, 1424],
 [3, 163],
 [4, 7019],
 [4, 1748],
 [4, 2395],
 [4, 2792],
 [4, 4832],
 [4, 113],
 [4, 5949],
 [4, 638],
 [4, 7866],
 [4, 5167],
 [5, 1748],
 [5, 638],
 [5, 2395],
 [5, 7019],
 [5, 2792],
 [5, 6702],
 [5, 3398],
 [5, 8546],
 [5, 5448],
 [5, 5008],
 [6, 1748],
 [6, 638],
 [6, 2395],
 [6, 2792],
 [6, 7019],
 [6, 113],
 [6, 4703],
 [6, 5167],
 [6, 7994],
 [6, 6702],
 [7, 6702],
 [7, 60],
 [7, 5554],
 [7, 4832],
 [7, 6880],
 [7, 8069],
 [7, 6748],
 [7, 1748],
 [7, 8546],
 [7, 2110],
 [8, 1748],
 [8, 638],
 [8, 2792],
 [8, 2395],
 [8, 5008],
 [8

In [ ]:
# Data Export
# 인덱스를 원래 데이터 상태로 변환하여 csv 저장합니다.
predict_list_idx = [[user2idx.index[user], item2idx.index[item]] for user, item in predict_list]
predict_df = pd.DataFrame(data=predict_list_idx, columns=['user', 'item'])
predict_df = predict_df.sort_values('user')

In [ ]:
predict_df

,user,item


## Required Package

- numpy==1.23.5
- pandas==1.5.3
- torch==2.1.0

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

